In [176]:
# Load libraries
library(vegan)
library(ape)
library(ggplot2)
library(dplyr)
library(RColorBrewer)


Warning message:
“package ‘RColorBrewer’ was built under R version 4.1.2”


In [168]:
# read in otu table
otu <- read.csv("shya_k31.sigs.compare.csv")

# Label the rows
rownames(otu) <- colnames(otu)


In [169]:
# open metadata file
cat_meta <- read.csv("meta.csv", sep=',', header=1, row.names=1)
# give sample IDs       
cat_meta$SampleID <- rownames(cat_meta)


In [170]:
# nmds
otu.nmds <- metaMDS(otu)
otu.nmds$stress

Run 0 stress 0.07694692 
Run 1 stress 0.08047023 
Run 2 stress 0.07744287 
... Procrustes: rmse 0.06493424  max resid 0.173473 
Run 3 stress 0.08487449 
Run 4 stress 0.07748032 
Run 5 stress 0.07694644 
... New best solution
... Procrustes: rmse 0.0002873923  max resid 0.001083886 
... Similar to previous best
Run 6 stress 0.07744301 
... Procrustes: rmse 0.06498052  max resid 0.1734918 
Run 7 stress 0.07694659 
... Procrustes: rmse 0.0001417456  max resid 0.0005361993 
... Similar to previous best
Run 8 stress 0.08083554 
Run 9 stress 0.0857182 
Run 10 stress 0.08042849 
Run 11 stress 0.07874386 
Run 12 stress 0.07874384 
Run 13 stress 0.07739947 
... Procrustes: rmse 0.05635371  max resid 0.1973494 
Run 14 stress 0.07874384 
Run 15 stress 0.07874387 
Run 16 stress 0.08080911 
Run 17 stress 0.0761916 
... New best solution
... Procrustes: rmse 0.01613867  max resid 0.07159446 
Run 18 stress 0.08083546 
Run 19 stress 0.07619148 
... New best solution
... Procrustes: rmse 0.0001700497  

[1] 0.07619148

In [171]:
# perform pcoa with ape package pcoa
pcoa <- pcoa(as.dist(otu))


# make a dataframe named axes, put pcoa values in there
axes <- as.data.frame(pcoa$vectors)

# Give df extra column with the rownames in it 
axes$SampleID <- rownames(axes)

# R will not automatically bind datapoints with the same name, but randomly bind them
# therefore order cat data with this
cat_meta.ordered <- cat_meta[match(row.names(otu.nmds$points), row.names(cat_meta)),]   


In [172]:
# calculate the eigenvalues for each pcoa axes 
eigval <- round(pcoa$values$Relative_eig * 100, digits = 2)

# merge those dfs
axes <- merge(cat_meta.ordered, axes, by.x = "SampleID", by.y = "SampleID")


In [173]:
# Put those eigenvalues in a df so they easy to get to. 
eigval <- data.frame( PC = 1:length(eigval), Eigval = eigval)
# head(eigval) # see top eigenvalues
eigval[[1,2]] # see first axes percentage
eigval[[2,2]] # second axes
eigval[[3,2]] # third axes
eigval[[4,2]] # fourth axes


[1] 9.72

[1] 6.35

[1] 4.98

[1] 4.74

In [175]:
# permanova
pmanova2 = adonis2(as.dist(otu) ~  treat, data = cat_meta.ordered)
pmanova2

In [185]:
# write to pdf
# pdf("PCOA.pdf")

# set plot
p <- ggplot(axes, aes(Axis.1, Axis.2), width = 7, height = 3) 

# set color of the points as the factor depth, shape as year, set size and see-throughness
p + geom_point(aes(colour=as.character(treat), size = 5,alpha=0.9, stroke=1)) +
 
  # set text for the axis lables (eigenvalues)
  xlab(paste("PCo1 (", eigval$Eigval[1], " %)", sep = "")) +  # or somthing else
  ylab(paste("PCo2 (", eigval$Eigval[2], " %)", sep = "")) +
  
  # set the colors of the points with the colorbrewer pallet
  scale_color_brewer(name = "Treatment", palette = "Dark2") +

#   # you can set colors manually by this: 
#   #scale_color_manual(name = 'soil', values = c("#377eb8", "#e41a1c", "yellow")) +

#   # set shapes of the points
#   #scale_shape_manual(name = "Plot", values=c(16,17,15,18)) +

  # tell where the legend has to be
  guides(color = guide_legend(title.position = "top", title.hjust = 0.5)) +
  theme_bw() +
  
  # set text size for whole graph. set the background color (white with no lines)
  theme(text = element_text(size = 20), panel.grid.major = element_blank(),
  panel.grid.minor = element_blank(),
        legend.position = "left") 
dev.off()